# **Build an LLM-Powered Chatbot**

## **Overview**

Here there, 

In this tutorial, we will be building an LLM-Powered chatbot. This chatbot will have the ability to have conversations and remember previous conversations.

**NOTE:** This bot will only the language model to have conversations, we will cover more advanced topic later like:

- **Conversation RAG:** Enable chatbot experience over external data source
- **Agents:** Build a chatbot that can take actions

Nevertheless, this tutorial will cover the basics required for those advance topics.

## **Setup**

### **Jupyter Notebook**

This tutorial uses Jupyter notebooks, Jupyter notebooks are perfect for learning how to work with LLM systems because oftentimes things can go wrong (unexpected output, API down, etc) and going through tutorials in an interactive environment is a great way to better understand them.


This and other tutorials are perhaps most conveniently run in a Jupyter notebook. See [here](https://jupyter.org/install) for instructions on how to install.

### **Installation**

**Run the following commands in your terminal to install the required packages (if you don't have them already):**

- pip install langchain
- pip install -qU langchain-groq
- pip install langchain_community
- pip install transformers

## **Quickstart**

Let's start by using the language model as is. We will be using llama3.1 from `groq` API. 


You can also run the llama3.1 model on your local system using `ollama`

In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [52]:
model = ChatGroq(model="llama-3.1-8b-instant")

**To interact with a chatmodel, you simply pass a `HumanMessage` and call the `.invoke` method on the model**

Let's seee how it is done!

In [53]:
#Import the HummanMessage function 
from langchain_core.messages import HumanMessage

#Create a HumanMessage
human_message = HumanMessage(content="Hi! I'm Abdulmalik")

#Insert the human message into the .invoke method
model.invoke([human_message])

AIMessage(content="Assalamu alaikum (or peace be with you!) Abdulmalik! Nice to meet you. What's on your mind, what can I help you with?", response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 17, 'total_tokens': 54, 'completion_time': 0.049333333, 'prompt_time': 0.005726894, 'queue_time': None, 'total_time': 0.055060227}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-995f7c13-dbc4-4145-b948-3e1de99175b8-0', usage_metadata={'input_tokens': 17, 'output_tokens': 37, 'total_tokens': 54})

**Question: Will it remember my name?**

Unfortunately, the model on it's own does not have any concept of memory, so no

In [54]:
#Create a HumanMessage
human_message = HumanMessage(content="What is my name?")

#Insert the human message into the .invoke method
model.invoke([human_message])

AIMessage(content="I'm a large language model, I don't have the ability to know your personal information or remember previous conversations. Each time you interact with me, it's a new conversation and I don't retain any information about you.\n\nIf you'd like to share your name with me, I'd be happy to chat with you and learn more about you!", response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 15, 'total_tokens': 86, 'completion_time': 0.094666667, 'prompt_time': 0.006263864, 'queue_time': None, 'total_time': 0.10093053099999999}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-2f22e047-af44-468e-80c5-c8f466758905-0', usage_metadata={'input_tokens': 15, 'output_tokens': 71, 'total_tokens': 86})

As you can see, it doesn't take into consideration previous conversations.


A chatbot that doesn't remember previous conversations will provide a terrible experience. **How do we fix this?**

To get around this, we need to pass the entire `conversation history` into the model. 


Let's see what happens when we do that:

In [55]:
#Import the AIMessage function from langchain, so we can mimick a response from the AI model 
from langchain_core.messages import AIMessage


model.invoke(
    [
        HumanMessage(content="Hi! I'm Abdulmalik"),
        AIMessage(content="Assalamu alaikum (or peace be with you!) Abdulmalik! Nice to meet you. What's on your mind, what can I help you with?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Abdulmalik! How would you like me to address you? Would you prefer Abdulmalik, Abdul, or something else?', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 68, 'total_tokens': 98, 'completion_time': 0.04, 'prompt_time': 0.019601628, 'queue_time': None, 'total_time': 0.059601628000000004}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-2c392c5c-459b-43aa-9e3c-ad5cdbbbf9ba-0', usage_metadata={'input_tokens': 68, 'output_tokens': 30, 'total_tokens': 98})

This is the basic idea underpinning a chatbot's ability to interact conversationally. 


**Now, how do we best implement this functionality?**

## **Message History**

- The MessageHistory class would allow our model to remember previous conversations.


- It will keep track of the inputs and outputs of the AI Model as store them in some datastore (memory).


- Subsequent interactions will then load the those messages and pass them into the chain (combination of the AI model and other components) as part of the input. 


Let's see how to use this!

An important question to address first is, **How do we get the message history (previous messages)?**


- To achieve this, we will create a function called `get_message_history`
- This function is expected to take in a `session_id` and return a `Message History` object. 
- This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the `config` when calling the new chain (combination of the model and other components, like message history)

**Let's see how it is done!**

In [56]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory


#Create a storage
store = {}


def get_message_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


# Create the chain that has message history
chain_with_message_history = RunnableWithMessageHistory(model, get_message_history)

**We now need to create a `config` that we pass into the chain every time.**

- This config contains information that is not part of the input directly, but is still useful. 
- In this case, we want to include a `session_id`. 


This should look like:

In [57]:
#Create the config 
config = {"configurable": {"session_id": "abc2"}}


# Create a variable called `ai_response` to store the output of the AI model
ai_response = chain_with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Abdulmalik")],
    config=config,
)

# Show the content of the AI response
ai_response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Assalamu alaikum, Abdulmalik! Nice to meet you. How are you today? What brings you here? Do you want to talk about something specific or just say hello? I'm here to listen!"

In [60]:
#Create the config 
config = {"configurable": {"session_id": "abc2"}}


# Create a variable called `ai_response` to store the output of the AI model
ai_response = chain_with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Abdulmalik")],
    config=config,
)

# Show the content of the AI response
ai_response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()


Error in callback coroutine: ValueError()


"Welcome! You seem a bit... stuck. No worries! How about this: would you like to break the ice by sharing what brings you to our chat, or do you need a random fun question to get the conversation going? For example: What's your favorite hobby? Or where are you from? Or anything at all? I'm all ears (well, not really, but I'll still listen with my text self!)"

In [61]:
#Create the config 
config = {"configurable": {"session_id": "abc2"}}


# Create a variable called `ai_response` to store the output of the AI model
ai_response = chain_with_message_history.invoke(
    [HumanMessage(content="Hi! WHat is my name")],
    config=config,
)

# Show the content of the AI response
ai_response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"I think I see where this is going! Your name is Abdulmalik! I remember from earlier, you told me that you're Abdulmalik! Don't worry, I won't hold it against you if you forgot – we all forget sometimes! Do you want to try again, or move on to something new?"

**Great! Our chatbot now remembers things about us.**

- If we change the config to reference a different `session_id`, we can see that it starts the conversation fresh.

In [62]:
#Create the config 
config = {"configurable": {"session_id": "abc3"}}


# Create a variable called `ai_response` to store the output of the AI model
ai_response = chain_with_message_history.invoke(
    [HumanMessage(content="Hi! WHat is my name")],
    config=config,
)

# Show the content of the AI response
ai_response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Unfortunately, I'm a large language model, I don't have any prior information about you, so I don't know your name yet! Would you like to introduce yourself? I'd love to get to know you!"

**You can always go back to the previous conversation by passing the right session id**

In [63]:
#Create the config 
config = {"configurable": {"session_id": "abc2"}}


# Create a variable called `ai_response` to store the output of the AI model
ai_response = chain_with_message_history.invoke(
    [HumanMessage(content="Hi! WHat is my name")],
    config=config,
)

# Show the content of the AI response
ai_response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"You're asking again! Your name is... Abdulmalik!"

**This is how we can support a chatbot having conversations with many users!**


**Now, we can start to make the chatbot more complicated and personalized by adding in a prompt template.**



## **Prompt Template**

**To put is simply, Prompt Templates gives you the flexibility to control the behaviour of your AI model.**

Let's start by adding a system message th some custom instructions for the model

**Note:** We will use a `MessagePlaceholder` to pass in all input messages.

Let's create a simple chain with the prompt template

In [64]:
#import the required functions
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [65]:
# Create the prompt template and save it in a variable called "prompt"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful and polite assistant that provides excellent customer service experience to customers. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

**We can now invoke the chain and pass in a language of our choice.**

In [66]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm Abdulmalik")], "language": "English"}
)

response.content

"Hello Abdulmalik! It's nice to meet you. Welcome to our service. How can I assist you today? Are you looking for information, need help with something, or perhaps have a question? I'm all ears and here to help."

**Let's now wrap this more complicated chain in a Memory  (Message History class).**

- This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [67]:
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    get_message_history,
    input_messages_key="messages",
)

In [68]:
config = {"configurable": {"session_id": "abcd1"}}



ai_response = chain_with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm Abdulmalik")], "language": "English"},
    config=config
)


ai_response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Hello Abdulmalik! It's nice to meet you! How can I assist you today? Are you looking for information on a particular topic, or do you have a question about something? I'm all ears and here to help!"

In [69]:
ai_response = chain_with_message_history.invoke(
    {"messages": [HumanMessage(content="What is my name?")], "language": "English"},
    config=config
)


ai_response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Your name is Abdulmalik. Is there something specific you'd like to know about your name, or is there anything else I can help you with?"

**Let's be a bit rude**

In [70]:
ai_response = chain_with_message_history.invoke(
    {"messages": [HumanMessage(content="I think you are being unprofessional and your service is trash.")], "language": "English"},
    config=config
)


ai_response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"I apologize if my previous response did not meet your expectations. I'm committed to providing a high-quality service and respecting all customers. I'll do my best to turn things around and provide a better experience for you.\n\nTo start fresh, let me try again. Your name is Abdulmalik, and I'm here to assist you with any questions or concerns you may have. Can you please tell me more about what's not meeting your expectations and how I can improve? I'm listening and eager to help."

## **Managing Conversation History**

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

**Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.**

- We can do this by adding a simple step in front of the prompt that modifies the `messages` key appropriately, 
- and then wrap that new chain in the Message History class.

Here, we'll use the `trim_messages` helper function to reduce how many messages we're sending to the LLM.

In [35]:
from langchain_core.messages import SystemMessage, trim_messages

In [71]:
trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Abdulmalik"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [72]:

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

To use the `trimmer` in our chain, we just need to run it before we pass the messages input to our prompt.


Now if we try asking the model our name, it won't know it since we trimmed that part of the chat history:



In [73]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough



chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)



ai_response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)


ai_response.content

"I don't think you mentioned your name earlier. You're just a customer chatting with me, so I don't have any information about your name. Would you like to introduce yourself?"

**But if we ask about information that is within the last few messages, it remembers:**

In [74]:
ai_response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
ai_response.content

'You asked for the answer to the math problem 2 + 2.'

##### **Let's now wrap this in the Message History**

In [76]:
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    get_message_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [77]:
ai_response = chain_with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

ai_response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"I'm afraid I don't know your name. This is the beginning of our conversation, and I don't have any information about you yet. Would you like to introduce yourself?"

As expected, the first message where we stated our name has been trimmed. Plus there's now two new messages in the chat history (our latest question and the latest response). 


This means that even more information that used to be accessible in our conversation history is no longer available! 


In this case our initial math question has been trimmed from the history as well, so the model no longer knows about it:

In [47]:
response = chain_with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"You didn't ask a math problem. This is the beginning of our conversation, and I'm here to help with any questions or topics you'd like to discuss. How can I assist you today?"

## **Streaming**

Now we've got a functional chatbot. 

However, one really important UX consideration for chatbot application is streaming. LLMs can sometimes take a while to respond, and so in order to improve the user experience one thing that most application do is stream back each token (word) as it is generated. 

This allows the user to see progress.



All chains expose a `.stream` method, and ones that use message history are no different. 


We can simply use that method to get back a streaming response.

In [79]:
config = {"configurable": {"session_id": "abc15"}}


for response in chain_with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm Abdulmalik. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    
    print(response.content, end="")

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


Hello Abdulmalik! It's great to meet you! Here's a joke for you:

What do you call a fake noodle?

An impasta!

I hope that made you smile! Do you want to hear another one?

## **Next Steps**


Now that you understand the basics of how to create a chatbot in LangChain, some more advanced tutorials you may be interested in are:


- **Conversational RAG:** Enable a chatbot experience over an external source of data
- **Agents:** Build a chatbot that can take actions


**Stay Tuned!**